In [1]:
# auto reload libraries (you do need to re-import libraries if you make changes)
%load_ext autoreload
%autoreload 2

# base 
import pandas as pd
import numpy as np
from pprint import pprint
import os

# preprocessing 
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import SimpleImputer, IterativeImputer,KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import PowerTransformer
from sklearn.decomposition import TruncatedSVD
# variable importance
from sklearn.inspection import permutation_importance

# models
from lightgbm import LGBMRegressor

# visualize
import matplotlib.pyplot as plt
import seaborn as sns

# own defined functions/classes 
from preprocessing.preprocess_pipe import MakeLowerCase
from preprocessing.preprocess_pipe import HdResolutionCategorizer
from preprocessing.preprocess_pipe import StorageCategorizer
from preprocessing.preprocess_pipe import SsdCategorizer

from preprocessing.preprocess_pipe import print_missing
from preprocessing.preprocess_pipe import calculate_perf
from preprocessing.preprocess_pipe import custom_scoring_func_single_p

from modelling.weight_samples import weights_samples

from postprocessing.postprocessing import plot_predictions_results
from postprocessing.postprocessing import plot_residuals

In [2]:
# global parameters
pd.set_option('display.max_columns', 500)
# interactive plotting
# %matplotlib widget
# run grid search
RUN_GRID_SEARCH = True
# style for plotting
plt.style.use('ggplot')
current_palette = sns.color_palette("colorblind")
sns.palplot(current_palette)
sns.set_palette("colorblind")

C:\Users\Pieter-Jan\Documents\KuLeuven\Semester2\AA\AAForB_Assignment2


# Step 1: import data and split in train and validation set
The validation set is more our own kind of test set

In [1]:
# read in trainig and validation data
df_all_train = pd.read_csv("../../../data/train.csv", sep=',')
df_test = pd.read_csv("../../../data/test.csv", sep=',')

print(f'Dimensions of all training data {df_all_train.shape}')
print(f'Dimension test data {df_test.shape}')

Dimensions of all training data (510, 22)
Dimension test data (222, 20)


In [4]:
# split in training and validation set
df_train = df_all_train.sample(frac=0.75, random_state=0, replace=False)
df_val = df_all_train.drop(df_train.index)

# reset index, if you don't resit missing rows get inserted in the pipeline
# see: https://stackoverflow.com/questions/31323499/sklearn-error-valueerror-input-contains-nan-infinity-or-a-value-too-large-for
df_train = df_train.reset_index().drop('index',axis=1)
df_val = df_val.reset_index().drop('index',axis=1)

# Step 2: define pipeline

In [5]:
#Numerical features to pass down the numerical pipeline 
numerical_features = ['screen_size' ,'pixels_x','pixels_y',
                      'ram', 'weight','ssd','storage']

#Categrical features to pass down the categorical pipeline 
categorical_features = ['brand','base_name', 'screen_surface','touchscreen',
                        'cpu','cpu_details','detachable_keyboard',
                        'discrete_gpu','gpu', 'os','os_details']

# define all unique features
features = np.unique(numerical_features).tolist() + np.unique(categorical_features).tolist()

# target variables
target = ['min_price','max_price']

In [6]:
# train
X_train = df_train[features]
y_train = df_train[target]

# validation (this is kind our own test set)
X_val  = df_val[features]
y_val = df_val[target]

# train_validation (this is all training data we have) for fitting the model
X_all_train = df_all_train[features]
y_all_train = df_all_train[target]

# test
X_test = df_test[features]

In [7]:
# you can add many more and 
# you can even define custom preprocessing steps like 'MakeLowerCase()'

# pipeline  uses only numerical features,
numeric_transformer = Pipeline(steps=[
    #('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
    #('imputer', KNNImputer(n_neighbors=5, weights='uniform',metric='nan_euclidean')),
    ('imputer', IterativeImputer(missing_values=np.nan, random_state=0, n_nearest_features=None, sample_posterior=True)),
    #('transformation', PowerTransformer(method='yeo-johnson',standardize=False)),
     #Scale features using statistics that are robust to outliers.
    ('scaler', RobustScaler(with_centering=True, with_scaling=True, quantile_range=(10.0, 90.0)))]) 

# pipeline use only categorical features
categorical_transformer = Pipeline(steps=[ 
    ('lowercase', MakeLowerCase()), # lower cases all columns containing strings
    #('sd_category' ,SsdCategorizer(drop_original_feature=True)),
    #('storage_category', StorageCategorizer(drop_original_feature=True)),
    ('imputer', SimpleImputer(missing_values=np.nan, strategy="most_frequent")),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# add both preprocessing pipelines in one pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])

In [8]:
# this is how the preporcessed pipeline looks like (just to have an idea)
pd.DataFrame(preprocessor.fit_transform(X_train)).shape

(382, 1)

# Step 3: add models to pipeline

Possible to fit multiple target variabels, so you **don't** need to fit a different models for min. price and max. price

### A) Training and parameter tuning

##### 1) Automatic tuning via grid search

I will only **do the tuning for the minimum price** and use the found parameters also for the maximum price

In [55]:
# define model: I just add some default parameters but you could
# also just write: RandomForestRegressor() since we will perform a grid search 
# to find good hyperparameter values
model = LGBMRegressor(random_state=0)

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(model.get_params())

# add to pipeline
# 1) min price
pipeline_min_p = Pipeline(memory=None,
              steps=[('preprocessor', preprocessor),
                     ('regressor', model)])


# add transformation on the target variable, by default power transformation 
# also performs standardization after performing the power transformation
# and back transform to the original space when outputting predictions 

# 1) min price
transformer_target = PowerTransformer(method='yeo-johnson',standardize=False)
scale_target = RobustScaler(with_centering=True, with_scaling=True, quantile_range=(10.0, 90.0))
pipeline_y = Pipeline(memory=None,
              steps=[('transformer', transformer_target),
                     ('scaler',scale_target)])

pipeline_min_p_update = TransformedTargetRegressor(regressor=pipeline_min_p, 
                                         transformer=pipeline_y)

Parameters currently in use:

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': 0,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}


In [92]:
# boosting_type : string, optional (default='gbdt')
boosting_type = ['gbdt', 'dart', 'goss']

learning_rate = [x for x in np.linspace(0.005, .25, num = 20)]

# The number of boosting stages to perform. 
# Gradient boosting is fairly robust to over-fitting so a large number usually results in better performance.
n_estimators = [int(x) for x in np.linspace(start = 500, stop = 3000, num = 20)]

# Number of samples for constructing bins.
subsample_for_bin = [int(x) for x in np.linspace(20, 500, num = 20)]

min_split_gain = [int(x) for x in np.linspace(0, .5, num = 20)]

min_child_samples = [int(x) for x in np.linspace(5, 50, num = 20)]

min_child_weight = [int(x) for x in np.linspace(0.001, 0.1, num = 20)]

colsample_bytree = [int(x) for x in np.linspace(0.1, 1, num = 20)]

# maximum depth of the individual regression estimators
max_depth = [int(x) for x in np.linspace(2, 50, num = 10)]

num_leaves= [int(x) for x in np.linspace(5, 100, num = 10)]
# L1 regul
reg_alpha = [int(x) for x in np.linspace(0, .1, num = 10)]

reg_lambda  = [int(x) for x in np.linspace(0, .1, num = 10)]

# Create the random grid
random_grid = {
   'regressor__regressor__boosting_type': boosting_type,
   'regressor__regressor__n_estimators': n_estimators,
   'regressor__regressor__learning_rate': learning_rate,
  # 'regressor__regressor__subsample_for_bin': subsample_for_bin,
   'regressor__regressor__min_child_samples': min_child_samples,
   'regressor__regressor__colsample_bytree': colsample_bytree,
   'regressor__regressor__min_split_gain': min_split_gain,
   # 'regressor__regressor__min_child_weight': min_child_weight,
   #'regressor__regressor__reg_alpha': reg_alpha,
   #'regressor__regressor__reg_lambda': reg_lambda,
   'regressor__regressor__max_depth': max_depth,
   'regressor__regressor__num_leaves':num_leaves
}

In [93]:
# define random search (and narrow down time grid search)

if RUN_GRID_SEARCH:
    min_p_random_search = RandomizedSearchCV(
       estimator = pipeline_min_p_update, 
       param_distributions = random_grid, n_iter = 1000,
       cv = 5, verbose=2, random_state=1, n_jobs = -1, refit=True,
       scoring=make_scorer(custom_scoring_func_single_p, greater_is_better=False)
    )

    # run grid search and refit with best hyper parameters
    weights_train_min_p =  weights_samples(df=y_train.iloc[:,0], order=0, plot_weights=False)
    min_p_random_search.fit(X_train, y_train.iloc[:,0])  
    print(min_p_random_search.best_params_)    
    print(min_p_random_search.best_score_)


# have look at the best hyperparameters and their respective performance (maybe also look at the sd)
pd.DataFrame(min_p_random_search.cv_results_).sort_values(
        by=['mean_test_score'],ascending=False).head(5)

Sum weights: 1.0
Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   14.1s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   42.1s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 3273 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done 4893 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 5000 out of 5000 | elapsed:  8.4min finished
C:\Users\Pieter-Jan\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:788: RuntimeWarning: invalid value encountered in subtract
  array_means[:, np.

{'regressor__regressor__num_leaves': 26, 'regressor__regressor__n_estimators': 1157, 'regressor__regressor__min_split_gain': 0, 'regressor__regressor__min_child_samples': 14, 'regressor__regressor__max_depth': 12, 'regressor__regressor__learning_rate': 0.017894736842105262, 'regressor__regressor__colsample_bytree': 1, 'regressor__regressor__boosting_type': 'gbdt'}
-150.14222780519


mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
146       1.116219      0.032716         0.045680        0.006597   
153       2.219085      0.035945         0.042089        0.007007   
107       1.127986      0.007192         0.031716        0.005997   
365       0.717279      0.014710         0.034708        0.006382   
160       6.811346      0.104205         0.078193        0.009368   

    param_regressor__regressor__num_leaves  \
146                                     26   
153                                     36   
107                                    100   
365                                     78   
160                                     47   

    param_regressor__regressor__n_estimators  \
146                                     1157   
153                                      763   
107                                      631   
365                                      500   
160                                     2078   

    param_regressor__regressor__min_split_gain  \
146                                          0   
153                                          0   
107                                          0   
365                                          0   
160                                          0   

    param_regressor__regressor__min_child_samples  \
146                                            14   
153                                            14   
107                                            23   
365                                            42   
160                                            14   

    param_regressor__regressor__max_depth  \
146                                    12   
153                                    12   
107                                    39   
365                                    23   
160                                    12   

    param_regressor__regressor__learning_rate  \
146                                 0.0178947   
153                                  0.172632   
107                                 0.0952632   
365                                  0.159737   
160                                 0.0436842   

    param_regressor__regressor__colsample_bytree  \
146                                            1   
153                                            1   
107                                            1   
365                                            1   
160                                            1   

    param_regressor__regressor__boosting_type  \
146                                      gbdt   
153                                      dart   
107                                      dart   
365                                      dart   
160                                      dart   

                                                params  split0_test_score  \
146  {'regressor__regressor__num_leaves': 26, 'regr...        -158.897127   
153  {'regressor__regressor__num_leaves': 36, 'regr...        -158.877406   
107  {'regressor__regressor__num_leaves': 100, 'reg...        -169.692399   
365  {'regressor__regressor__num_leaves': 78, 'regr...        -160.188322   
160  {'regressor__regressor__num_leaves': 47, 'regr...        -162.850340   

     split1_test_score  split2_test_score  split3_test_score  \
146        -158.566219        -132.060030        -143.539602   
153        -162.753068        -135.014015        -154.441036   
107        -151.112689        -134.838963        -147.767617   
365        -151.662950        -142.012890        -150.918884   
160        -163.166604        -132.112735        -150.492896   

     split4_test_score  mean_test_score  std_test_score  rank_test_score  
146        -157.648162      -150.142228       10.719249                1  
153        -153.392415      -152.895588        9.542934                2  
107        -161.466107      -152.975555       11.925361                3  
365        -160.153380      -152.987285        6.777571                4  
160    

##### 2) Manual parameters tuning

In [96]:
# train your final model on all data with best parameters 

# 1) min price
model = LGBMRegressor(
    boosting_type='gbdt',
    n_estimators=1157,
    learning_rate =0.0178947,
    min_child_samples=14,
    min_split_gain=0,
    colsample_bytree=1,
    max_depth=12,
    num_leaves = 26,
    random_state=0
    
)


# add to pipeline

# 1) min price
pipeline_min_p = Pipeline(memory=None,
              steps=[('preprocessor', preprocessor),
                     ('regressor', model)],
              verbose=True)

# 2) min price
pipeline_max_p = Pipeline(memory=None,
              steps=[('preprocessor', preprocessor),
                     ('regressor', model)],
              verbose=True)

# again add transformer for target variable

# 1) min price
pipeline_min_p_update = TransformedTargetRegressor(regressor=pipeline_min_p, 
                                         transformer=pipeline_y)
# 2) max price
pipeline_max_p_update = TransformedTargetRegressor(regressor=pipeline_max_p, 
                                         transformer=pipeline_y)

# fit final model on all training data we have at hand

# 1) min price
weights_train_min_p = weights_samples(y_train.iloc[:,0], order=0)
pipeline_min_p_update.fit(X_train, y_train.iloc[:,0])

# 2) max price
weights_train_max_p = weights_samples(y_train.iloc[:,1], order=0)
pipeline_max_p_update.fit(X_train, y_train.iloc[:,1])

Sum weights: 1.0
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.1s
[Pipeline] ......... (step 2 of 2) Processing regressor, total=   0.8s
Sum weights: 1.0
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.2s
[Pipeline] ......... (step 2 of 2) Processing regressor, total=   0.9s


TransformedTargetRegressor(check_inverse=True, func=None, inverse_func=None,
                           regressor=Pipeline(memory=None,
                                              steps=[('preprocessor',
                                                      ColumnTransformer(n_jobs=None,
                                                                        remainder='drop',
                                                                        sparse_threshold=0.3,
                                                                        transformer_weights=None,
                                                                        transformers=[('num',
                                                                                       Pipeline(memory=None,
                                                                                                steps=[('imputer',
                                                                                                        Itera

In [97]:
# performance on data where the model was fit one (should be very low)
pred_train_min_p = pipeline_min_p_update.predict(X_train)
pred_train_max_p = pipeline_max_p_update.predict(X_train)

# calculate performance
pred_train = pd.DataFrame([pred_train_min_p,pred_train_max_p]).T
calculate_perf(y_train, pred_train)

{'minimum price': 57.43002055563661,
 'maximum price': 56.296107171417844,
 'total error': 113.72612772705446}

##### Performance on validation data

In [98]:
# performance on validation data
pred_val_min_p = pipeline_min_p_update.predict(X_val)
pred_val_max_p = pipeline_max_p_update.predict(X_val)

# calculate performance 
pred_val = pd.DataFrame([pred_val_min_p,pred_val_max_p]).T
calculate_perf(y_val, pred_val)

{'minimum price': 117.34538654789144,
 'maximum price': 116.9530964073627,
 'total error': 234.29848295525414}

In [99]:
# save predictions on validation data
# submission format
submission_format_validation = pd.DataFrame.from_dict(
 {'ID':df_val['id'].values,
 'MIN':pred_val_min_p,
 'MAX':pred_val_max_p}).set_index('ID')

# write to csv
submission_format_validation.to_csv('output/validation/light_gbm.csv' ,
                            header=True, index=True, sep=',')

### C) Post processing

In [100]:
fig, axs = plt.subplots(2, 2, figsize=(9, 7))
axs = np.ravel(axs)

# fitted against true predictions minimum price
plot_predictions_results(ax=axs[0], 
                        y_true=y_val.iloc[:,0], 
                        y_pred=pred_val_min_p, 
                        title="Boosting Min. Price Test Set", 
                        log_scale=True)

# fitted against true predictions maximum price
plot_predictions_results(ax=axs[1], 
                        y_true=y_val.iloc[:,1], 
                        y_pred=pred_val_max_p, 
                        title="Boosting Max. Price Test Set", 
                        log_scale=True)

# residuals minimum price
plot_residuals(ax=axs[2], 
               y_true=y_val.iloc[:,0], 
               y_pred=pred_val_min_p, 
               title="", 
               log_scale=False,
               order=1)

# residuals maximum price
plot_residuals(ax=axs[3], 
               y_true=y_val.iloc[:,1], 
               y_pred=pred_val_max_p, 
               title="", 
               log_scale=False,
               order=1)
fig.tight_layout()
plt.savefig('output/figures/light_gbm/fig1.png', dpi=1000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [101]:
plt.figure(figsize=(8,4))
plt.plot(pred_val_max_p - pred_val_min_p, label="Predictions Test", linestyle="--")
plt.plot(y_val.iloc[:,1] - y_val.iloc[:,0], label="Truth Test", linestyle=':')
plt.xlabel("Observations")
plt.ylabel("Max. Price - Min. Price")
plt.legend(loc="upper right");
plt.savefig('output/figures/light_gbm/fig2.png', dpi=1000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [102]:
vip_val_min_p = permutation_importance(pipeline_min_p_update, X_val, y_val.iloc[:,0], n_repeats=10,
                                random_state=1, n_jobs=3)

vip_val_max_p = permutation_importance(pipeline_min_p_update, X_val, y_val.iloc[:,1], n_repeats=10,
                                random_state=1, n_jobs=3)

sorted_idx_min_p = vip_val_min_p.importances_mean.argsort()
sorted_idx_max_p = vip_val_max_p.importances_mean.argsort()

In [107]:
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
ax = np.ravel(ax)
# minimum price
ax[0].boxplot(vip_val_min_p.importances[sorted_idx_min_p].T,
           vert=False, labels=X_val.columns[sorted_idx_min_p])
ax[0].set_title("Permutation Importances: Min. Price (Test set)", fontsize=9)
ax[0].xaxis.set_tick_params(labelsize=8)
ax[0].yaxis.set_tick_params(labelsize=8)

# maximum price
ax[1].boxplot(vip_val_min_p.importances[sorted_idx_max_p].T,
           vert=False, labels=X_val.columns[sorted_idx_max_p])
ax[1].set_title("Permutation Importances: Max. Price (Test set)", fontsize=9)
ax[1].xaxis.set_tick_params(labelsize=8)
ax[1].yaxis.set_tick_params(labelsize=8)
fig.tight_layout()
plt.show()
plt.savefig('output/figures/light_gbm/fig3.png', dpi=1000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### D) Predictions test data

In [104]:
# train your final model on all data with best parameters 


model_final = LGBMRegressor(
    boosting_type='gbdt',
    n_estimators=1157,
    learning_rate =0.0178947,
    min_child_samples=14,
    min_split_gain=0,
    colsample_bytree=1,
    max_depth=12,
    num_leaves = 26,
    random_state=0
)


# add to pipeline

# 1) min price
pipeline_min_p_final = Pipeline(memory=None,
              steps=[('preprocessor', preprocessor),
                     ('regressor', model_final)],
              verbose=True)

# 2) min price
pipeline_max_p_final = Pipeline(memory=None,
              steps=[('preprocessor', preprocessor),
                     ('regressor', model_final)],
              verbose=True)

# again add transformer for target variable

# 1) min price
pipeline_min_p_final = TransformedTargetRegressor(regressor=pipeline_min_p_final, 
                                         transformer=pipeline_y)
# 2) max price
pipeline_max_p_final = TransformedTargetRegressor(regressor=pipeline_max_p_final, 
                                         transformer=pipeline_y)

# fit final model on all training data we have at hand

# 1) min price
weights_all_train_min_p = weights_samples(y_all_train.iloc[:,0], order=2)
pipeline_min_p_final.fit(X_all_train, y_all_train.iloc[:,0])

# 2) max price
weights_all_train_max_p = weights_samples(y_all_train.iloc[:,1], order=2)
pipeline_max_p_final.fit(X_all_train, y_all_train.iloc[:,1])

Sum weights: 1.0
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.1s
[Pipeline] ......... (step 2 of 2) Processing regressor, total=   1.0s
Sum weights: 1.0
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.2s
[Pipeline] ......... (step 2 of 2) Processing regressor, total=   0.9s


TransformedTargetRegressor(check_inverse=True, func=None, inverse_func=None,
                           regressor=Pipeline(memory=None,
                                              steps=[('preprocessor',
                                                      ColumnTransformer(n_jobs=None,
                                                                        remainder='drop',
                                                                        sparse_threshold=0.3,
                                                                        transformer_weights=None,
                                                                        transformers=[('num',
                                                                                       Pipeline(memory=None,
                                                                                                steps=[('imputer',
                                                                                                        Itera

In [105]:
# performance on all data where the model was fit one (should be very low)
pred_all_train_min_p = pipeline_min_p_final.predict(X_all_train)
pred_all_train_max_p = pipeline_max_p_final.predict(X_all_train)

# calculate performance
pred_all_train = pd.DataFrame([pred_all_train_min_p, pred_all_train_max_p]).T
calculate_perf(y_all_train, pred_all_train)

{'minimum price': 48.42264183468457,
 'maximum price': 49.526094510301775,
 'total error': 97.94873634498634}

In [106]:
# make predictions on test data
pred_test_min_p = pipeline_min_p_final.predict(X_test)
pred_test_max_p = pipeline_max_p_final.predict(X_test)

# submission format
submission_format = pd.DataFrame.from_dict(
 {'ID':df_test['id'].values,
 'MIN':pred_test_min_p,
 'MAX':pred_test_max_p}).set_index('ID')

# write to csv
submission_format.to_csv('output/submission/light_gbm.csv' ,
                            header=True, index=True, sep=',')